# 2-4-3 LoRA

[LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)

## 0 章节目标

* 理解 LoRA 的基本思想
* 学习如何使用 Huggingface 的代码微调自己的模型
* 了解 LoRA 在其他 LLM 中的应用

## 1 问题背景

为什么仅用几千条样本就能将一个数十亿参数的模型微调得比较好？以往的一些结果（如 [Qin et al.](https://arxiv.org/abs/2012.13255) 和 [Aghajanyan et al.](https://arxiv.org/abs/2012.13255)）显示，尽管预训练模型的参数量很大，但每个下游任务对应的本征维度（Intrinsic Dimension）并不大，换句话说，理论上我们可以微调大模型中很小一部分的参数，就能很好地适应下游任务。

何为本征维度？本征维度是指达到任务期望性能所需的最小参数量。如果把大模型看作是将数据映射到高维空间进行处理的一种模型，那么某一个具体的细分任务，可能只需要用到这一高维空间的某个子空间。我们无法找到解决这些问题的最佳的子空间，但可以手动选取某些参数作为子空间。当我们对该子空间中的参数进行微调时，能够达到全量参数微调的一定水平，如 [Aghajanyan et al.](https://arxiv.org/abs/2012.13255) 论文中选择的 $90\%$ 的性能，那么这一子空间满足该下游任务的要求。所有符合要求的子空间中，维度最小的那个子空间对应的维度为该任务的本征维度，记为 $d$。

假定模型的全量参数为 $\theta^D$，其中本征维度上的参数为 $\theta^d$，那么针对某个特定下游任务的微调可以表示为：

$$\theta^D=\theta^D_0+\theta^d M$$

其中矩阵 $M$ 将本征维度上的参数向量 $\theta^d\in\mathbb{R}^d$ 映射到全量参数空间 $\mathbb{R}^D$ 中。这是大模型信息压缩的观点。

LoRA 的思想正是基于这样的观点，认为模型在微调 adaptation 的时候也具有这样的内在秩，参数的增量可以被低秩分解，只需要优化低秩分解的矩阵即可。

## 2 微调方法

对于预训练的参数矩阵 $W_0^\mathbb{R}^{m\times n}$，LoRA 不直接微调 $W_0$，而是对增量作低秩分解：

$$W = W_0 + A B,\quad A\in\mathbb{R}^{m\times r},B\in\mathbb{R}^{r\times n}$$

其中 $A,\,B$ 其一使用全零初始化。由于本征维度很小的结论，低秩矩阵的 $r$ 可以取的很小，甚至为 $1$，因此需要被优化的参数量大大降低了。

![](https://img-blog.csdnimg.cn/img_convert/06b30661c111c0cae25adbaa9c0e3819.png#averageHue=#eb9245&clientId=ue4258a53-1a5a-4&from=paste&height=376&id=u6bc01447&originHeight=453&originWidth=496&originalType=binary&ratio=2&rotation=0&showTitle=false&size=25068&status=done&style=none&taskId=u1f08ffa2-cd94-46bf-baea-46a2cf18e48&title=&width=412)

LoRA 在微调预训练参数的子集这种大模型微调范式的基础上进一步减少了约束，不要求在微调过程中累积梯度更新的权重矩阵具有满秩，而随着 LoRA 可微调参数逐渐增大，直至微调全部参数，LoRA 低秩矩阵的秩也收敛于预训练模型参数矩阵的秩。换句话说，随着可训练参数的增加，LoRA 收敛于全量参数模型，而基于 Adapter 的微调方案收敛于 MLP，基于 Prefix Tuning 的方案可接收的输入序列则越来越小。

需要注意的是，LoRA 和其他很多微调方案一样，只是降低了权重，但没有降低计算量。关于这一点，我们只需要观察低秩矩阵 $A,\,B$ 的梯度：

$$\frac{\partial \mathcal{L}}{\partial A} = \frac{\partial \mathcal{L}}{\partial W} B^{\top},\quad \frac{\partial \mathcal{L}}{\partial B} = A^{\top}\frac{\partial \mathcal{L}}{\partial W}$$

在训练过程中，LoRA 更新所用的梯度 $\frac{\partial \mathcal{L}}{\partial A},\,\frac{\partial \mathcal{L}}{\partial B}$ 来自全量更新的梯度 $\frac{\partial \mathcal{L}}{\partial W}$，因此从理论上来说 LoRA 的计算量没有减少。但是 LoRA 与其他参数高效的微调方案一样，都可以通过冻结模型大部分层、以及更新参数量减少所带来的多卡通信效率提升来提高实际上的训练速度。与 Adapter、Prefix-Tuning 等方案类似地，LoRA 在不同任务之间切换的成本同样很低，对于几十 G 的大模型来说，LoRA 的低秩矩阵模块可能只有几 M。LoRA 的独有优势如上面所说，是一种更加直观，更接近全量微调的微调方式。此外在推理过程中，$AB$ 可以视为单个矩阵，这也可以减少推理成本。